__Plot DENISE seismograms__

Daniel Köhn 
Kiel, 02/03/2018

__Import Libraries__

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from matplotlib.colors import LightSource, Normalize
from matplotlib.pyplot import gca
from pylab import rcParams
from matplotlib import rc
from matplotlib.ticker import FormatStrFormatter

__Define fonts__

In [ ]:
FSize = 20
font = {'color':  'black',
        'weight': 'normal',
        'size': FSize}
mpl.rc('xtick', labelsize=FSize) 
mpl.rc('ytick', labelsize=FSize) 
rcParams['figure.figsize'] = 16, 12

__Define parameters__

In [ ]:
ROT_COMP = 0 # rotate components

__Read seismograms (SPECFEM-2D)__

In [ ]:
basename_sem = "../NEXD-2D/lambs_meshpy/ref/"
state = "AA"

In [ ]:
# define file names for all components
name_seis_1_x = basename_sem + "S0001." + state + ".BXX.semd"
name_seis_1_z = basename_sem + "S0001." + state + ".BXZ.semd"

In [ ]:
# read 2-component data
data_sem_1_x = np.genfromtxt(name_seis_1_x, dtype=None)
data_sem_1_z = np.genfromtxt(name_seis_1_z, dtype=None)

In [ ]:
# define time vector [s]
tsem = data_sem_1_x[:,0]

__Read seismograms (analytical solution)__

In [ ]:
basename_analy = "../NEXD-2D/lambs_meshpy/ref/"

In [ ]:
# define file names for all components
name_seis_1_x = basename_analy + "Ux_file_ascii_rec1"
name_seis_1_z = basename_analy + "Uz_file_ascii_rec1"

In [ ]:
# read 2-component data
data_analy_1_x = np.genfromtxt(name_seis_1_x, dtype=None)
data_analy_1_z = np.genfromtxt(name_seis_1_z, dtype=None)

In [ ]:
# define time vector [s]
tanaly = data_analy_1_x[:,0]

__Read seismograms (DENISE)__

In [ ]:
basename_nexd = "su/lamb_DH_1m/DENISE_lamb"

In [ ]:
# define file names for all components
name_seis_1_x = basename_nexd + "_x.ascii.shot1"
name_seis_1_z = basename_nexd + "_y.ascii.shot1"

In [ ]:
# read 2-component data
data_denise_1_x = np.genfromtxt(name_seis_1_x, dtype=None)
data_denise_1_z = np.genfromtxt(name_seis_1_z, dtype=None)

In [ ]:
# define time vector [s]
# dt = 1.1e-4   # DH = 0.5 m
dt = 2.2e-4   # DH = 1 m
# dt = 4.4e-4   # DH = 2 m
# dt = 8.8e-4   # DH = 4 m
ns = len(data_denise_1_x)
tdenise = np.arange(dt, dt*(ns+1), dt)

In [ ]:
# extimate ux/uz from DENISE vx/vz solutions by numerical integration
data_denise_1_ux = dt * np.cumsum(data_denise_1_x)
data_denise_1_uz = dt * np.cumsum(data_denise_1_z)

In [ ]:
# rotate components
if (ROT_COMP==1):
    angle = 10.0 * np.pi / 180.0
    denise_1_ux_rot = np.cos(angle) * data_denise_1_ux + np.sin(angle) * data_denise_1_uz
    denise_1_uz_rot = -np.sin(angle) * data_denise_1_ux + np.cos(angle) * data_denise_1_uz
    
if (ROT_COMP==0):
    denise_1_ux_rot = data_denise_1_ux
    denise_1_uz_rot = data_denise_1_uz

In [ ]:
# shift time vector of SEM and analytical solution to t=0
tanaly += np.abs(np.min(tanaly))
tsem += np.abs(np.min(tsem))

__Define SubPlot__

In [ ]:
def do_plot(n, seis_sem, seis_denise, seis_analy, an, title):
    
    ax=plt.subplot(2, 1, n)    
    
    rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
    rc('text', usetex=True)
    
    plt.plot(tsem,seis_sem[:,1]/np.amax(seis_sem[:,1]))
    plt.plot(tdenise,-seis_denise/np.amax(-seis_denise))
    plt.plot(tanaly,seis_analy[:,1]/np.amax(seis_analy[:,1]))
    
    #plt.plot(tsem,seis_sem[:,1])
    #plt.plot(tdenise,seis_denise)
    #plt.plot(tanaly,seis_analy[:,1])
        
    # legend
    plt.legend(('SPECFEM-2D', 'DENISE (DH = 1 m)', 'analytical'), shadow=True, fontsize=FSize)
    
    a = gca()
    a.set_xticklabels(a.get_xticks(), font)
    a.set_yticklabels(a.get_yticks(), font)
    plt.axis('tight')
    plt.title(title, fontdict=font)
    plt.ylabel(an, fontdict=font)
    if n!=2:
        ax.set_xticks([])
    if n==2:
        plt.xlabel('Time [s]', fontdict=font)
    plt.tight_layout()

__Plot SubPlots__

In [ ]:
plt.close('all')
plt.figure()
do_plot(1, data_sem_1_x, denise_1_ux_rot, data_analy_1_x, '$U_x$ []', "Lambs problem (receiver 1)")
do_plot(2, data_sem_1_z, denise_1_uz_rot, data_analy_1_z, '$U_z$ []', "")
filename = "lambs_problem_receiver_1_u_DH_1m.pdf"
plt.savefig(filename, bbox_inches='tight', format='pdf')
#plt.savefig(filename, bbox_inches='tight', format='png', dpi=200)
plt.show()